In [1]:
"""
Procedure for calibrating generative models using the unconditional Sig-Wasserstein metric.
"""
import ml_collections
import copy
import wandb
import yaml
import os

from os import path as pt
from src.evaluations.evaluations import compute_discriminative_score, fake_loader, compute_classfication_score, plot_summary, full_evaluation
import torch
from src.utils import get_experiment_dir, save_obj
from torch import nn

/home/jtao/ac3/envs/test_metric/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_dir = 'configs/' + 'train_vae.yaml'
with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))
# print(config)
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu_id
print(os.environ["CUDA_VISIBLE_DEVICES"])
# Set the seed
# torch.manual_seed(config.seed)
# np.random.seed(config.seed)

# initialize weight and bias
# Place here your API key.
# setup own api key in the config
os.environ["WANDB_API_KEY"] = config.wandb_api
tags = [
    config.algo,
    config.dataset,
]

wandb.init(
    project='Generative_model_evaluation',
    config=copy.deepcopy(dict(config)),
    entity="deepintostreams",
    tags=tags,
    group=config.dataset,
    name=config.algo
    # save_code=True,
    # job_type=config.function,
)
config = wandb.config
print(config)
if (config.device ==
        "cuda" and torch.cuda.is_available()):
    config.update({"device": "cuda:0"}, allow_val_change=True)
else:
    config.update({"device": "cpu"}, allow_val_change=True)
# torch.cuda.set_per_process_memory_fraction(0.5, 0)get_dataset
from src.datasets.dataloader import get_dataset
train_dl, test_dl = get_dataset(config, num_workers=4)
from src.baselines.models import get_trainer
trainer = get_trainer(config, train_dl, test_dl)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1


wandb: Currently logged in as: hangl (deepintostreams). Use `wandb login --relogin` to force relogin


{'D_hidden_dim': 64, 'D_num_layers': 3, 'D_out_dim': 5, 'D_steps_per_G_step': 3, 'G_hidden_dim': 64, 'G_input_dim': 5, 'G_num_layers': 2, 'algo': 'TimeVAE', 'batch_size': 256, 'comment': None, 'conditional': False, 'custom_seas': None, 'data_type': 'angles', 'dataset': 'ROUGH', 'device': 'cuda', 'gpu_id': '1', 'hidden_layer_sizes': [64, 64], 'latent_dim': 64, 'lr_G': 0.002, 'model': 'TimeVAE', 'model_type': 'VAE', 'n_lags': 50, 'num_classes': 3, 'num_gen_seas': 0, 'pretrained': False, 'reconstruction_wt': 3.0, 'seed': 0, 'steps': 2000, 'train': True, 'trend_poly': 0, 'use_residual_conn': True, 'use_scaler': False, 'wandb_api': '0a2ae01d4ea2b07b7fca1f71e45562ab1a123c80'}
data shape: torch.Size([256, 50, 2])
TimeVAE
ROUGH_TimeVAE
13
25
49
VAE: VariationalAutoencoderConvInterpretable(
  (encoder): encoder_block(
    (encoder): Sequential(
      (conv_0): Conv1d(5, 64, kernel_size=(3,), stride=(2,), padding=(1,))
      (norm_0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [4]:
get_experiment_dir(config)

WARNING! The model exists in directory and will be overwritten


In [6]:
import datetime

print(datetime.datetime.now())
trainer.fit(config.device)

2022-10-17 15:44:20.611335


  0%|                                                                                                              | 0/2000 [00:00<?, ?it/s]


NameError: name 'x_real_batch' is not defined

In [7]:
import torch

In [8]:
x = torch.rand([4,2,1])

In [11]:
x.permute([0,2,1]).shape

torch.Size([4, 1, 2])

In [10]:
x.shape

torch.Size([4, 2, 1])